In [ ]:
import sys
from math import gcd
from bisect import bisect_left


class FenwickTree:
    __slots__ = ("tree", "size")

    def __init__(self, size: int):
        self.size = size
        self.tree = [0] * (size + 1)

    def add(self, index: int, delta: int) -> None:
        index += 1
        while index <= self.size:
            self.tree[index] += delta
            index += index & -index

    def query(self, index: int) -> int:
        index += 1
        result = 0
        while index > 0:
            result += self.tree[index]
            index -= index & -index
        return result

    def range_query(self, left: int, right: int) -> int:
        if left > right:
            return 0
        return self.query(right) - self.query(left - 1)


def calculate_answer(n: int, a: list[int], l: int, r: int, k: int) -> int:
    if not hasattr(calculate_answer, "prepared"):
        calculate_answer.prepared = True

        values = a[:]
        length = n

        if length == 1:
            calculate_answer.answers = {}
            return 0

        for i in range(length - 1):
            values[i] = abs(values[i] - values[i + 1])

        length -= 1
        values = values[:length]

        queries = calculate_answer.queries
        queries.sort(key=lambda x: x[2], reverse=True)

        events = {}
        current_intervals = []

        for i in range(length - 1, -1, -1):
            current_gcd = values[i]
            current_right = i
            next_intervals = []

            for prev_gcd, prev_right in current_intervals:
                new_gcd = gcd(current_gcd, prev_gcd)
                if new_gcd != current_gcd:
                    next_intervals.append((current_gcd, current_right))
                    current_gcd = new_gcd
                    current_right = prev_right
                else:
                    current_right = prev_right

            next_intervals.append((current_gcd, current_right))
            current_intervals = next_intervals

            for gcd_value, right in current_intervals:
                key = gcd_value if gcd_value != 0 else 10**9
                events.setdefault(key, []).append((i, right))

        sorted_gcds = sorted(events.keys(), reverse=True)
        next_pos = list(range(-1, length - 1))

        fenwick = FenwickTree(length)
        for i in range(length):
            fenwick.add(i, i - 1)

        answers = [0] * len(queries)
        gcd_pointer = 0

        for left, right, k_value, query_index in queries:
            while gcd_pointer < len(sorted_gcds) and sorted_gcds[gcd_pointer] >= k_value:
                value = sorted_gcds[gcd_pointer]
                for position, right_limit in events[value]:
                    old = next_pos[position]
                    if right_limit > old:
                        next_pos[position] = right_limit
                        fenwick.add(position, right_limit - old)
                gcd_pointer += 1

            pos = bisect_left(next_pos, right, left, right + 1)
            part1 = fenwick.range_query(left, pos - 1)
            part2 = (right - pos + 1) * right
            count = right - left + 1
            arithmetic_sum = count * ((left - 1) + (right - 1)) // 2
            answers[query_index] = part1 + part2 - arithmetic_sum

        calculate_answer.answers = answers

    idx = calculate_answer.query_index
    calculate_answer.query_index += 1
    return calculate_answer.answers[idx]


class FastInput:
    def __init__(self):
        self.stdin = sys.stdin

    def read_line(self):
        return sys.stdin.readline().strip()

    def read_tokens(self):
        return self.read_line().split()

    def read_int(self):
        return int(self.read_line())

    def read_ints(self):
        return map(int, self.read_tokens())


def solution():
    input = FastInput()

    n, q = map(int, input.read_tokens())
    a = list(input.read_ints())

    calculate_answer.queries = []
    calculate_answer.query_index = 0

    for i in range(q):
        l, r, k = map(int, input.read_tokens())
        calculate_answer.queries.append((l - 1, r - 2, k, i))

    for _ in range(q):
        print(calculate_answer(n, a, 0, 0, 0), end=" ")